In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.regularizers import l2
from keras.initializers import he_normal

import tensorflow as tf
from keras.callbacks import LearningRateScheduler

print(f"Tensorflow {tf.__version__}")

Tensorflow 2.20.0


In [2]:
# Function to load the CIFAR-10 dataset
import pickle
from keras.utils import to_categorical
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_cifar_10_data(data_dir, negatives=False):
    """
    Return train_data, train_filenames, train_labels, test_data, test_filenames, test_labels
    """

    # get the meta_data_dict
    # num_cases_per_batch: 1000
    # label_names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # num_vis: :3072

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    # training data
    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, to_categorical(cifar_train_labels), \
        cifar_test_data, cifar_test_filenames, to_categorical(cifar_test_labels), cifar_label_names



In [ ]:
def resnet_v1_eembc():
    
    inputs = tf.keras.Input(shape = (32, 32, 3))
    x = tf.keras.layers.Conv2D(
        16, 
        kernel_size=(5,5),
        activation='relu',
        padding = 'same',
        use_bias=False)(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(
        64,
        kernel_size=(5,5),
        activation='relu',
        padding = 'same',
        use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(
        64,
        kernel_size=(5,5),
        activation='relu',
        padding = 'same',
        use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(
        10,
        activation='softmax',
        use_bias=False)(x)

    # Create functional model
    model= tf.keras.Model(inputs, x)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [ ]:
train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = load_cifar_10_data('cifar-10-batches-py')

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.


In [5]:
model = resnet_v1_eembc()

# Train model
history = model.fit(
    train_data, train_labels,       # training data
    batch_size=128,                  # batch size
    epochs=20,                       # Maximum number of epochs
    validation_split=0.1,   # Percentage of training data used for validation
)

import os
print("Before saving:", os.getcwd())
model.save("cnn1.keras")


# Test model
predictions_keras = model.predict(test_data, verbose=0)
test_loss, test_accuracy = model.evaluate(test_data, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("ResNetCIFAR.keras")

2025-10-14 09:48:47.898665: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 16)     │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        25,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 64)       │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        10,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,440 (544.69 KB)

 Trainable params: 139,440 (544.69 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 64s 169ms/step - accuracy: 0.2012 - loss: 2.1786 - val_accuracy: 0.2856 - val_loss: 2.0195
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 57s 102ms/step - accuracy: 0.2994 - loss: 1.9544 - val_accuracy: 0.3280 - val_loss: 1.9150
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 35s 99ms/step - accuracy: 0.3542 - loss: 1.8156 - val_accuracy: 0.3974 - val_loss: 1.7091
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 32s 90ms/step - accuracy: 0.3863 - loss: 1.7140 - val_accuracy: 0.4240 - val_loss: 1.6171
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 42s 93ms/step - accuracy: 0.4165 - loss: 1.6291 - val_accuracy: 0.4416 - val_loss: 1.5765
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 33s 93ms/step - accuracy: 0.4401 - loss: 1.5697 - val_accuracy: 0.4514 - val_loss: 1.5477
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 40s 114ms/step - accuracy: 0.4583 - loss: 1.5216 - val_accuracy: 0.4794 - val_loss: 1.4474
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 39s 111ms/step - accuracy: 0.4741 - loss: 

In [6]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [7]:
net = snn_converter.load_keras_model("ResNetCIFAR.keras", show_info=True)

* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 16) 
* MaxPooling2D layer: max_pooling2d, (16, 16, 16) 
* Conv2D layer: conv2d_1, (16, 16, 64) 
* MaxPooling2D layer: max_pooling2d_1, (8, 8, 64) 
* Conv2D layer: conv2d_2, (8, 8, 64) 
* MaxPooling2D layer: max_pooling2d_2, (4, 4, 64) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense, 10 
    weights: (10, 1024)
    mean -5.4935295338509604e-05, std 0.05212739482522011
    min -0.1603514701128006, max 0.20855838060379028



In [8]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [15:48<00:00,  3.16s/it]


In [16]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Convert one-hot labels (returned by load_cifar_10_data) to integer class indices
y_true = test_labels[:300]
if y_true.ndim == 2 and y_true.shape[1] > 1:
    y_true = np.argmax(y_true, axis=1)

print(classification_report(y_true, predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(y_true, predictions_snn))

              precision    recall  f1-score   support

           0       0.69      0.50      0.58        36
           1       0.59      0.79      0.68        24
           2       0.60      0.22      0.32        27
           3       0.36      0.45      0.40        29
           4       0.42      0.48      0.45        23
           5       0.42      0.29      0.34        28
           6       0.49      0.88      0.63        34
           7       0.56      0.37      0.44        27
           8       0.70      0.57      0.63        37
           9       0.64      0.77      0.70        35

    accuracy                           0.54       300
   macro avg       0.55      0.53      0.52       300
weighted avg       0.56      0.54      0.53       300

Test accuracy of the SNN: 0.5433333333333333


In [12]:
model = tf.keras.models.load_model('ResNetCIFAR.keras')

loss, accuracy = model.evaluate(test_data, test_labels, verbose=1)




313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5887 - loss: 1.1669


In [13]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(y_true, predictions_snn))

Test accuracy of the ANN: 0.5887
Test accuracy of the SNN: 0.5433333333333333
